In [61]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Fonctions utiles

In [62]:
def f_html(lien):
    return requests.get(lien).text.encode("utf-8")

def f_soup(lien):
    return BeautifulSoup(f_html(lien),'lxml')

def f_convertdict(lst):
    res_dct = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)}
    return res_dct

def delete_several_cha_from_end(string,n):
    for i in range(n):
        string = string.rstrip(string[-1])
    return string

def delete_byindex(string,index):
    if len(string) > index:
        return string[0 : index : ] + string[index + 1 : :]



Récupérer nom et lien des rayons

In [63]:
def search_rayons(lien,list_rayons_principal=[]):
    acceuil = f_soup(lien)
    rayon_tag = acceuil.find_all('div',class_='navigation-node trigger-sublevel not-loaded')

    for rayon in rayon_tag:

        #search nom rayon principal
        rayon_principal = rayon.text.strip().replace(' ','').replace(",","")

        #search lien rayon principal
        rayon_lien= rayon.a['href']
        rayon_lien =f'{lien}{rayon_lien}'

        #ajout nom et lien des rayons principales dans une liste
        dict_rayon_principal={
            'rayon principal' : rayon_principal,
            'lien rayon' : rayon_lien 
        }
        list_rayons_principal.append(dict_rayon_principal)

    return list_rayons_principal

Récupérer noms des sous rayons

In [64]:
def search_sous_rayon (lien,lienp ='https://www.auchan.fr',list_sous_rayon=[]):
    soup = f_soup(lien)
    rayon_sous_tag = soup.find_all('div', class_='universe-block')
    for rayon_sous in rayon_sous_tag:

        #search nom sous rayon
        nom_rayon_sous = rayon_sous.find('div', class_='universe-block__title')
        nom_rayon_sous = nom_rayon_sous.text.strip().replace(' ','').replace(',','')
        
        #search lien sous rayon
        lien_sous_rayon = rayon_sous.a['href']
        lien_sous_rayon =f'{lienp}{lien_sous_rayon}'
        
        #ajout nom et lien des sous rayons dans une liste
        dict_rayon_sous={
            'sous rayon' : nom_rayon_sous,
            'lien sous rayon' : lien_sous_rayon
        }
        
        list_sous_rayon.append(dict_rayon_sous)

    return list_sous_rayon

Assemble list rayon principal et sous rayon

In [65]:
def assemble_rayonp_rayons(lien):
    #initialize dataframe 
    df_all_rayon = pd.DataFrame(columns = ['rayon principal', 'lien rayon', 'sous rayon', 'lien sous rayon'])
 
    #search rayons principal
    df_rayon_principal = pd.DataFrame(search_rayons(lien))

    for i,lien_rayon in enumerate(df_rayon_principal['lien rayon']):

        #search sous rayons
        list_rayon_sous = search_sous_rayon(lien_rayon)

        #si pas de sous rayons
        if len(list_rayon_sous) == 0:

            #ajout N/A dans column sous rayons du dataframe
            list_rayon = [{
                'rayon principal' : df_rayon_principal.iloc[i]['rayon principal'],
                'lien rayon' : df_rayon_principal.iloc[i]['lien rayon'],
                'sous rayon' : 'N/A',
                'lien sous rayon' :'N/A'}]
            df_rayon = pd.DataFrame(list_rayon)
            df_all_rayon = pd.concat([df_all_rayon,df_rayon],ignore_index=True)

        #si il y des sous rayons
        else : 

            #ajout de chaque sous rayons dans dataframe
            for k,sous_rayon in enumerate(list_rayon_sous):

                df_sous_rayon = pd.DataFrame([sous_rayon])
    
                list_rayon = [{
                    'rayon principal' : df_rayon_principal.iloc[i]['rayon principal'],
                    'lien rayon' : df_rayon_principal.iloc[i]['lien rayon'],
                    'sous rayon' : df_sous_rayon.iloc[0]['sous rayon'],
                    'lien sous rayon' : df_sous_rayon.iloc[0]['lien sous rayon']}]
                df_rayon = pd.DataFrame(list_rayon)
                df_all_rayon = pd.concat([df_all_rayon,df_rayon],ignore_index=True)
                
        list_rayon_sous.clear()

    return df_all_rayon
'''
lien ='https://www.auchan.fr'
df_all_rayon = assemble_rayonp_rayons(lien)
#df_all_rayon.to_csv("list_all_rayon.csv")
print(df_all_rayon)
'''


'\nlien =\'https://www.auchan.fr\'\ndf_all_rayon = assemble_rayonp_rayons(lien)\n#df_all_rayon.to_csv("list_all_rayon.csv")\nprint(df_all_rayon)\n'

Récupérer produits

In [66]:

def search_produits(lien):
    
    #initialize dataframe 
    df_all_produits = pd.DataFrame(columns = ['rayon principal', 'lien rayon', 'sous rayon', 'lien sous rayon','nom produits'])
    
    #search tous sous rayons
    df_all_rayon = assemble_rayonp_rayons(lien)
    
    for i,lien_sous_rayon in enumerate(df_all_rayon['lien sous rayon']):

        #si pas de lien sous rayon
        if lien_sous_rayon =='N/A':

            #ajout N/A dans column produits du dataframe
            list_produits = [{
                'rayon principal' : df_all_rayon.iloc[i]['rayon principal'],
                'lien rayon' : df_all_rayon.iloc[i]['lien rayon'],
                'sous rayon' : 'N/A',
                'lien sous rayon' :'N/A',
                'nom produits' : 'N/A'
                }]
            df_produits = pd.DataFrame(list_produits)
            df_all_produits = pd.concat([df_all_produits,df_produits],ignore_index=True)
        
        #si il y a un lien sous rayon
        else :
            
            #chercher les produits du lien sous rayon
            acceuil = f_soup(lien_sous_rayon)
            produits = acceuil.find_all('div',class_="product-thumbnail__details")

            #ajout de chaque produit dans le dataframe
            for produit in produits:

                produit = produit.text.replace(' ','').replace('\n',',')
                
                list_produits = [{
                    'rayon principal' : df_all_rayon.iloc[i]['rayon principal'],
                    'lien rayon' : df_all_rayon.iloc[i]['lien rayon'],
                    'sous rayon' : df_all_rayon.iloc[i]['sous rayon'],
                    'lien sous rayon' : df_all_rayon.iloc[i]['lien sous rayon'],
                    'nom produits' : produit
                    }]
                df_produits = pd.DataFrame(list_produits)
                df_all_produits = pd.concat([df_all_produits,df_produits],ignore_index=True)

    return df_all_produits


lien ='https://www.auchan.fr'
df_all_produits = search_produits(lien)
df_all_produits.to_csv("df_tous_produits.csv")

